In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *
from packages.restaurant.mulProcess_restaurant_scraping_proxy import *

import os
import glob
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 

from multiprocessing import Pool
from functools import partial

In [2]:
# *** following this step to use multiprocessing method for restaurant scraping with remote scraping browser
# - select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
# - find out which is the last page of a province in wongnai.com (manually)

# ** select a province by index of 'ALL_REGION_ID_WONGNAI' (manually)
Idx_of_region = 63
cur_region_data = ALL_REGION_ID_WONGNAI[Idx_of_region]

# ** find out which is the last page of a province in wongnai.com (manually)
last_page_number = 281

#
cur_province_en = cur_region_data[0]
cur_province_th = cur_region_data[1]
cur_regionId = cur_region_data[2]

# create list of : [1, 2, 3, ...., last_page]
# page_number_list = np.arange(last_page_number) + 1
page_number_list = np.arange(30) + 1
print("check page number list: ")
print(page_number_list)

# get list of dataframe result of all restaurants in a province
# res_allRestaurants_list = []
with Pool(processes=13) as pool: 
    print("a 1")
    func_param_fix = partial(mulProcess_helper_scrape_restaurants_by_province, province = cur_province_th, wongnai_regionId = cur_regionId)
    print("a 2")
    res_allRestaurants_list = pool.map(func_param_fix, page_number_list)
    print("end process for province --> %s " % (cur_province_th))

# concat all dataframe into a final one to save in csv.
res_allRestaurants_df = pd.DataFrame()
check_page = 1
print("check len of list result dataframe --> ", len(res_allRestaurants_list))
for cur_df in res_allRestaurants_list:
    print("cur_df --> page : ", check_page)
    check_page += 1
    print("len cur_df --> ", len(cur_df))
    res_allRestaurants_df = pd.concat([res_allRestaurants_df, cur_df])
    # print("res_allRestaurants_df :")
    # print(res_allRestaurants_df)

# print("last check res :")
# print(res_allRestaurants_df)
# remove duplicate restaurant 
res_allRestaurants_df.drop_duplicates(subset=['name', 'sub_name'], inplace=True)
# set new index
res_allRestaurants_df.set_index(['name', 'sub_name'], inplace=True)

# save result dataframe to .csv
res_file_name = 'res_restaurant_%s.csv' % (cur_province_en)
res_path = os.path.join(fh.STORE_RESTAURANT_SCRAPING, 'res_restaurant_scraping', res_file_name) 
res_allRestaurants_df.to_csv(res_path, encoding="utf-8")
    

check page number list: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
a 1
a 2
end process for province --> มุกดาหาร 
check len of list result dataframe -->  30
cur_df --> page :  1
len cur_df -->  10
cur_df --> page :  2
len cur_df -->  10
cur_df --> page :  3
len cur_df -->  10
cur_df --> page :  4
len cur_df -->  10
cur_df --> page :  5
len cur_df -->  10
cur_df --> page :  6
len cur_df -->  0
cur_df --> page :  7
len cur_df -->  10
cur_df --> page :  8
len cur_df -->  10
cur_df --> page :  9
len cur_df -->  10
cur_df --> page :  10
len cur_df -->  10
cur_df --> page :  11
len cur_df -->  10
cur_df --> page :  12
len cur_df -->  10
cur_df --> page :  13
len cur_df -->  5
cur_df --> page :  14
len cur_df -->  1
cur_df --> page :  15
len cur_df -->  10
cur_df --> page :  16
len cur_df -->  10
cur_df --> page :  17
len cur_df -->  10
cur_df --> page :  18
len cur_df -->  10
cur_df --> page :  19
len cur_df -->  10
cur_df --> page :  20
len